# Урок 1. Метрики качества классификации ч.1

В уроке мы разбирали матрицу ошибок и некоторые метрики, основанные на терминах матрицы ошибок. Здесь же посмотрим, как эти метрики применять на практике и что  по ним можно выяснить.

### 1.1
Выкачайте тренировочный датасет Titanic - train.csv - с сайта [kaggle](https://www.kaggle.com/c/titanic/data). С помощью функции pd.read_csv() загрузите данные в датафрейм. Выведите первые 20 строк и проанализируйте данные: какие колонки присутствуют (более конкретная информация по ним есть на сайте kaggle), каким образом в них обозначены данные и какие типы данных используются (используйте pandas.dtypes). 

In [100]:
import pandas as pd

In [101]:
df = pd.read_csv('train.csv')
df.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [103]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

### 1.2
Проверьте, имеются ли пропущенные значения в колонках, и выведите сумму всех пропущенных значений в каждой из колонок.

In [104]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### 1.3
Замените все пропущенные значения колонки Age на медианы в зависимости от пола человека: т.е. если пол человека в строке с пропущенным значением "male", заменяете пропущенное значение возраста на медиану по всем известным возрастам мужчин, и наоборот. Выведите медианы возраста в зависимости от пола. Пропущенные значения колонок Cabin и Embarked замените на U (Unknown).

Примечание: для группировки по полу можно использовать метод df.groupby(), который имеет встроенные агрегатные функции (в т.ч. для вычисления медианы).  Для установки пропущенных значений в датасете воспользуйтесь функцией df.apply().

In [105]:
df[['Cabin', 'Embarked']] = df[['Cabin', 'Embarked']].fillna('U')

In [106]:
median_ages = df.groupby('Sex')['Age'].median()
df['Age'] = df.apply(lambda x: median_ages[x['Sex']] if pd.isna(x['Age']) else x['Age'], axis=1)

In [107]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

### 1.4
Выведите возраст пассажиров с PassengerID = [6, 20]. Убедитесь, что заполнены все пропущенные значения (воспользуйтесь функцией df.isnull() ).

In [108]:
df_temp = df.loc[df['PassengerId'].isin(range(6, 21))]['Age']
df_temp

5     29.0
6     54.0
7      2.0
8     27.0
9     14.0
10     4.0
11    58.0
12    20.0
13    39.0
14    14.0
15    55.0
16     2.0
17    29.0
18    31.0
19    27.0
Name: Age, dtype: float64

In [109]:
df_temp.isnull()

5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
Name: Age, dtype: bool

Так как не все классификаторы могут работать с категориальными признаками, в данном случае необходимо заменить их на числовые.

Также можно было бы выделить расширенные признаки, такие как статус пассажира (Mr., Mrs., Miss., Dr., Master. и т.д.), собственный индекс билета, индекс палубы и другие, но здесь ограничимся выбором более простых признаков.

### 1.5
В колонке Sex замените значения на 0, если пол "male", и на 1, если "female". В колонке Embarked замените параметры "U", "S", "C", "Q" на 0, 1, 2, 3 соответственно. Отбросьте колонки PassengerId, Name, Ticket, Cabin. Выведите первые 20 строк получившегося набора данных.

In [110]:
df['Sex'] = df['Sex'].map({'female': 1, 'male': 0}) 
df['Embarked'] = df['Embarked'].map({'U': 0, 'S': 1, 'C': 2, 'Q': 3})

In [111]:
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis='columns', inplace=True)

In [112]:
df.head(20)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,1
1,1,1,1,38.0,1,0,71.2833,2
2,1,3,1,26.0,0,0,7.9250,1
3,1,1,1,35.0,1,0,53.1000,1
4,0,3,0,35.0,0,0,8.0500,1
5,0,3,0,29.0,0,0,8.4583,3
6,0,1,0,54.0,0,0,51.8625,1
7,0,3,0,2.0,3,1,21.0750,1
8,1,3,1,27.0,0,2,11.1333,1
9,1,2,1,14.0,1,0,30.0708,2


In [113]:
df.to_csv('Titanic_prepared.csv')

### 1.6
Приступим к построению классификаторов. **Условимся, что, если функция или объект модели имеют параметр random_state, то устанавливаем его равным 17 в каждом из случаев.**

Разделите данные на тренировочный и тестовый датасеты, установив размер тестового как 0.25 (первая колонка Survived является целевой, поэтому необходимо сначала ее отделить от признаков).

In [64]:
from sklearn.model_selection import train_test_split 

In [66]:
target = df['Survived']
X = df.drop(['Survived'], axis='columns')

In [68]:
x_train, x_test, y_train, y_test = train_test_split(X, 
                                                    target,
                                                    test_size=0.25,
                                                    random_state=17)

In [71]:
target.sum()

342

In [72]:
len(target)

891

### 1.7
Обучите на полученных выборках несколько классификаторов, которые импортированы ниже.

**Примечание: в методе ближайших соседей используйте количество соседей, равное 5.**

In [73]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [76]:
knn = KNeighborsClassifier(n_neighbors=5)
knn_model = knn.fit(x_train, y_train)

nb = GaussianNB()
nb_model = nb.fit(x_train, y_train)

dtc = DecisionTreeClassifier(random_state=17)
dtc_model = dtc.fit(x_train, y_train)

lr = LogisticRegression()
lr_model = lr.fit(x_train, y_train)

Очевидно, в этом случае использовать метрику accuracy, которую мы применяли до этого, не совсем правильно: она не даст верную оценку классификатору, потому как выживших явно меньше, чем погибших.

Вспомним про такие метрики как точность и полнота. Здесь они подойдут хорошо, так как у нас имеются положительный и отрицательный классы, и определить корректно один из них может оказаться более важным .

### что использовать
лучше мы определим, что человек выживет, хотя он на самом деле умер, чем наоборот. Используем Recall

### 1.8
В каждом из пунктов следующего теста выберите, что важнее максимизировать: точность (precision) или полноту (recall). Выпишите ответы.

1. Вероятность того, что при определенной поломке самолета он сможет долететь до пункта назначения (1 - долетел, 0 - не долетел).
2. Предсказание, представляет ли человек опасность, по анализу психического состояния (1 - представляет опасность, 0 - не представляет опасности).
3. Предсказание ухода клиента (1 - клиент ушел, 0 - остался).
4. Выявление рака на основе медицинских показателей (1 - болен раком, 0 - здоров).
5. Предсказание летальности при наблюдаемой мутации (1 - выживание, 0 - летальный исход).
6. Определение важности происшествия для экстренных служб (1 - важно, 0 - неважно).
7. Окупятся ли вложения в бизнес (1 - окупятся, 0 - не окупятся).

### От себя:
Presision - (все больные - 5%) максимизируем, когда важно не назначить 1 тому, у кого 0. Лучше назначить 0 единице, чем наоборот.
Recall - (все больные - 100%) максимизируем, когда лучше назначить единицу нулю, чем пропустить эту 1, и назначить 0 при единице

Ваш ответ:

1. Presision (лучше сказать, что не долетит, когда мог бы, чем наоборот)<br>
2. Recall (С точки зрения общества в целом лучше сказать, что представляет опасность тот, кто не представляет. Но для человека последствия такой ошибки значительны =))<br>
3. Непонятны цели предсказания. но если считать, что в того клиента, который уйдёт менеджер не вкладывается. То лучше он будет вкладываться в того, кто в итоге уйдёт (то есть назначили 0 единице). А значит Presision<br>
4. Recall (лучше сказать, что болен раком, когда не болен, чем наоборот)<br>
5. Presision (если сказал, что выживет, то это должно быть точно. Лучше сказать, что умрет тому, кто выживет, чем наоборот)<br>
6. Recall (Лучше сказать, что важно не важное, чем пропустить важное)<br>
7. Presision (Если считаем, что вложения окупятся, то это должно быть наверняка (нельзя назначить единицу нулю))<br>

### 1.9
Определите, что важнее использовать в случае датасета Titanic: полноту или точность. Объясните, почему.

### 1.10
Для каждого классификатора выведите матрицу ошибок и самостоятельно рассчитайте метрику recall (расчеты должны присутствовать). Проверьте расчеты, воспользовавшись встроенной метрикой recall_score.

### ответ на 1.9 - полноту, Recall.
Например, мы ищем выживших. И пусть лучше мы умершего будем считать выжившим, чем наоборот.

In [78]:
from sklearn.metrics import confusion_matrix, recall_score

the count of true negatives is $C_{00}$, false negatives is $C_{10}$, true positives is $C_{11}$ and false positives is $C_{01}$. <br>
recall: TP/(TP + FN)

In [83]:
def my_recall(conf_matrix):
    return conf_matrix[1][1]/(conf_matrix[1][1] + conf_matrix[1][0])

In [85]:
y_pred = knn.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[110,  21],
       [ 47,  45]], dtype=int64)

In [91]:
print(f'my recall = {my_recall(cm)}, recall_score = {recall_score(y_test, y_pred)}')

my recall = 0.6956521739130435, recall_score = 0.6956521739130435


In [92]:
y_pred = nb.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[111,  20],
       [ 27,  65]], dtype=int64)

In [93]:
print(f'my recall = {my_recall(cm)}, recall_score = {recall_score(y_test, y_pred)}')

my recall = 0.7065217391304348, recall_score = 0.7065217391304348


In [94]:
y_pred = dtc.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[112,  19],
       [ 26,  66]], dtype=int64)

In [95]:
print(f'my recall = {my_recall(cm)}, recall_score = {recall_score(y_test, y_pred)}')

my recall = 0.717391304347826, recall_score = 0.717391304347826


In [96]:
y_pred = lr.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[114,  17],
       [ 28,  64]], dtype=int64)

In [97]:
print(f'my recall = {my_recall(cm)}, recall_score = {recall_score(y_test, y_pred)}')

my recall = 0.6956521739130435, recall_score = 0.6956521739130435


### 1.11
Выберите ту модель, на которой метрика recall давала лучший ответ, и рассчитайте для нее precision, используя встроенную функцию.

In [98]:
from sklearn.metrics import precision_score

In [99]:
y_pred = dtc.predict(x_test)
precision_score(y_test, y_pred)

0.7764705882352941